# Model Testing

## Import Model and Dataset

In [15]:
# Model and dataset
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_path = "fake_news_bert_model_full_freeze"

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Load CSVs
fake_df = pd.read_csv(r"C:\Users\colby\OneDrive\Desktop\School\Capstone\FakeNewsDataset.csv", usecols=["title", "text"])
real_df = pd.read_csv(r"C:\Users\colby\OneDrive\Desktop\School\Capstone\RealNewsDataset.csv", usecols=["title", "text"])

# Add labels
fake_df["label"] = 0
real_df["label"] = 1

# Combine and shuffle
df = pd.concat([fake_df, real_df]).sample(frac=1).reset_index(drop=True)

# Combine title and text
df["content"] = df["title"] + " " + df["text"]
df = df[["content", "label"]]



## Testing Dataset

In [16]:
# Split into train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["content"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

In [17]:
def predict_fake_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(-1).item()
    return predicted_class

In [18]:
def prepare_input(title, text):
    return f"{title} {text}"

In [19]:
real_title = "NASA Announces Successful Launch of Artemis I"
real_text = "The uncrewed mission marks the beginning of a new era in lunar exploration. NASA's Artemis I successfully launched from Kennedy Space Center."
real_input = prepare_input(real_title, real_text)
print("Prediction (Real Example):", predict_fake_news(real_input))


Prediction (Real Example): 1


In [20]:
fake_title = "Aliens Land in New York, Greet Citizens with Pizza"
fake_text = "Reports confirm that a UFO landed in Central Park. Witnesses say the aliens offered pepperoni pizza as a sign of peace."
fake_input = prepare_input(fake_title, fake_text)
print("Prediction (Fake Example):", predict_fake_news(fake_input))

Prediction (Fake Example): 1


In [21]:
def count_tokens(text):
    return len(tokenizer.tokenize(text))

print("Tokens in real_input:", count_tokens(real_input))
print("Tokens in fake_input:", count_tokens(fake_input))

Tokens in real_input: 35
Tokens in fake_input: 34


In [25]:
for i in range(25):
    sample = val_texts[i]
    label = val_labels[i]
    prediction = predict_fake_news(sample)
    print(f"\nSample #{i + 1}")
    print("Actual Label:", "Real" if label == 1 else "Fake")
    print("Predicted Label:", "Real" if prediction == 1 else "Fake")


Sample #1
Actual Label: Real
Predicted Label: Real

Sample #2
Actual Label: Real
Predicted Label: Real

Sample #3
Actual Label: Real
Predicted Label: Real

Sample #4
Actual Label: Real
Predicted Label: Real

Sample #5
Actual Label: Fake
Predicted Label: Real

Sample #6
Actual Label: Real
Predicted Label: Real

Sample #7
Actual Label: Fake
Predicted Label: Real

Sample #8
Actual Label: Fake
Predicted Label: Fake

Sample #9
Actual Label: Fake
Predicted Label: Fake

Sample #10
Actual Label: Fake
Predicted Label: Fake

Sample #11
Actual Label: Real
Predicted Label: Real

Sample #12
Actual Label: Real
Predicted Label: Real

Sample #13
Actual Label: Real
Predicted Label: Real

Sample #14
Actual Label: Real
Predicted Label: Real

Sample #15
Actual Label: Real
Predicted Label: Real

Sample #16
Actual Label: Fake
Predicted Label: Real

Sample #17
Actual Label: Real
Predicted Label: Real

Sample #18
Actual Label: Fake
Predicted Label: Fake

Sample #19
Actual Label: Fake
Predicted Label: Real

S

In [23]:
def prepare_input(title, text=""):
    return f"{title} {text}" if text else title


In [43]:
test_cases = {
    "Real": [
        "Pope Francis launched aggressive reforms to eliminate budget deficits and reduce spending. He assembled a high-level commission and slashed salaries and subsidies. Despite resistance, his efforts transformed Vatican financial transparency.",
        "Iconic Oregon Coast town voted one of the best and most historic spots in US Astoria was ranked on two of USA Today's 2025 10Best Readers’ Choice Awards for best small towns and most historic small towns in the western United States. USA Today asked a group of industry experts to nominate their favorite small towns in the United States based on places that exhibit the best in travel, food, drink and lifestyle before presenting them to the public to vote on.",
        "Border Protection agents detain 8 workers at dairy farm owned by Olympic runner's family Eight farmworkers were detained by U.S. Customs and Border Protection agents on Monday in Franklin County, Vermont, according to a spokesperson from the organization Migrant Justice. According to Migrant Justice, the arrests happened Monday afternoon. NBC5 learned that the workers who were arrested were employees at Pleasant Valley Farms in Berkshire, according to Amanda St. Pierre. The farm is owned by Mark and Amanda St. Pierre and their sons, Bradley and Jamie. They are related by marriage to Olympic runner Elle St. Pierre.",
        "U.S. Economy Adds 250,000 Jobs in March, Unemployment Steady at 3.8%",
        "UNICEF Launches Campaign to Provide Clean Water in Sub-Saharan Africa",
        "Biden Signs Bipartisan Infrastructure Bill into Law",
        "Tesla Reports Record Q4 Revenue as EV Demand Surges",
        "Olympics 2024 to Be Hosted in Paris with Enhanced Sustainability Focus",
        "Supreme Court weighs California’s power to ban gas cars The Supreme Court on Wednesday will hear oral arguments in a high-stakes battle over California’s authority to impose stricter vehicle emissions rules than the rest of the country, a case now shaped by a Trump administration push to roll back state-led climate initiatives. At issue is whether a coalition of energy producers and royalty owners, including Diamond Alternative Energy, can sue over the Environmental Protection Agency’s 2022 decision to grant California’s Clean Air Act waiver. The waiver, reinstated under former President Joe Biden, allows the state to implement stricter vehicle emissions standards to boost the electric vehicle industry. ",
        "Thousands queue to see Pope Francis’ body ahead of funeral The body of Pope Francis is lying in state in St. Peter’s Basilica, where it will remain for three days until his funeral Saturday, expected to be attended by world leaders including US President Donald Trump. His body was transferred to the basilica during a procession earlier Wednesday, and was followed by a service led by Cardinal Kevin Farrell, who holds the position of “camerlengo” (or chamberlain) tasked with making arrangements for the funeral and conclave in the weeks ahead. Francis’ coffin was laid at the Altar of the Confessio, a sacred space in front of the main tomb of St. Peter, the first pope. Cardinals in the basilica approached to view the coffin in pairs, followed shortly after by the first members of the public who paid their respects. The basilica opened to the public at 11 a.m. local time (5 a.m. ET) Wednesday and will remain open until midnight. On Thursday and Friday, it will open at 7 a.m. and close at midnight."
    ],
    "Fake": [
        "‘Severance’ Creator Says Next Season Of Show Will Solely Be Released As TV Recaps On Vulture.com LOS ANGELES—Saying the new format would help him present the series as he had always intended, Severance creator Dan Erickson announced Monday that the TV show’s next season would be released solely as episode recaps on Vulture.com. “After some debate in the writers’ room, we concluded that 500-word write-ups in New York magazine’s pop culture section were the perfect way to continue unfolding the mystery of Lumon Industries and its severed employees,” said Erickson, admitting that it was a creative risk, but that reading the “outstanding work” already put out by Vulture‘s renowned freelancers as they summarized the show’s earlier seasons gave him confidence this would be a change for the better. “From here on out, every Friday you’ll have to turn to Vulture to get your Severance fix. Trust us here. Once you read a few of these season-three recaps, you’ll realize you won’t miss cinematography, music, lighting, sets, Ben Stiller’s directing, or even actors bringing to life the story of Mark, Gemma, Dylan, Mr. Milchick, and Helly R. All you need is the incredible work of this entertainment news website. Plus, these will have a little unanswered question section at the end, which is always fun.” Erickson added that it was difficult to part ways with Apple TV+, but Vulture had simply made the better offer.",
        "RFK Jr. Flushes Nation’s Antidepressants WASHINGTON—Declaring that there was no better time for the U.S. populace to go cold turkey, Health and Human Services Secretary Robert F. Kennedy Jr. reportedly attempted to flush the nation’s antidepressants this week. “Listen, I found these in your medicine cabinet,” said Kennedy, the 71-year-old Cabinet member stunning Americans in all 50 states as he stood over the toilet flushing millions of prescriptions for fluoxetine, escitalopram, sertraline, and paroxetine down the bowl. “You don’t need these—they were only holding you back. Nuh-uh-uh, Vermont, no buts about it. Just push through the next few days, and you’ll see exactly what I’m talking about. If you still feel bad after that, try some fish oil. Trust me. Your life is about to be so much better.” At press time, reports confirmed Kennedy was frantically plunging the overflowing toilet.",
        "God Too Obsessed With Ants Right Now To Focus On Next Pope THE HEAVENS—Admitting that He had barely even noticed the leader of the Catholic Church had died, God, our Lord and Heavenly Father, announced Tuesday that He was too obsessed with ants right now to focus on the next pope. “While I want to commit to finding a successor to Pope Francis, I’m currently in kind of an ant phase right now, and it’s taking up my whole life,” said the Creator of All Things, explaining that ever since He discovered how cool the insects were, He let His duties overseeing the papal conclave fall by the wayside. “Here’s the thing. Ants look small, but they are actually super strong and can lift over 50 times their own body weight. Plus, they communicate with pheromones! I know the Catholic Church is in mourning and needs a strong, compassionate leader, blah, blah, blah, but seriously—did you know that ants actually grow their own fungus? They’re tiny little farmers. How cool is that?” Later, after reportedly suffering an unexpected ant bite, God confirmed that the next pope would be selected on the basis of how effectively he could kill insects.",
        "Even Investigators Not Sure Why They Keep Releasing Details Of Gene Hackman’s Death SANTA FE, NM—Appearing visibly giddy as they released to the public a new barrage of information about the deceased couple’s living conditions, personal lives, and private health issues, local authorities announced Friday that even they were not sure why they kept sharing details of Gene Hackman and his wife Betsy Arakawa’s deaths. “We have no idea why, but every time we find a new detail about their tragic passing, I’m like, ‘Oh my God, we have to tell everyone!’” said Ben Chavez of the Santa Fe County Sheriff’s Office, admitting that he felt compelled by a force he couldn’t understand to continue painting a timeline of the couple’s last moments, despite investigators having known for nearly two months that both died of natural causes with no evidence of foul play. “She and one of the dogs were dead for days—did we tell you that? As soon as the medical examiner said ‘hantavirus,’ it was like I physically could not hold the words in my mouth. By the way, did we show you these pictures of the rat nests? Gross, right? See, we can’t stop! We released hours of footage, and I don’t think 10 minutes passed before all these images of their cluttered, messy home were published online with a salacious headline. What a thrill! Hey, want to know what was in the fridge?” At press time, Chavez was reportedly investigating whether or not innocuous security footage of Arakawa grocery shopping the day before she likely died could be spun into evidence that she was a devil-worshipping cannibal.",
        "Financial Experts Recommend Diversifying Portfolio With Multiple Harebrained Schemes NEW YORK—Claiming it was the only way to protect one’s assets against economic volatility, a group of financial experts recommended this week that Americans diversify their portfolios with multiple harebrained schemes. “Rather than rely on a single half-baked investment, we strongly encourage people to have several cockamamie business plans to fall back on,” said J.P. Morgan financial advisor Jonathan Rattler, adding that it was important to mitigate risk through a combination of foolhardy ventures like breeding koi fish, stealing copper wire from a scrapyard, and buying a machine on Alibaba that makes lab-grown diamonds. “We suggest investing in a variety of dubious sectors, including miracle hair-growth supplements, alpaca rentals, and using outlets at public libraries to run your own bitcoin farm. Also, we know a guy who makes $2 million a year installing vending machines in local prisons. If you want, we can put you in touch.” Rattler added that Americans planning to retire by age 60 should really consider giving their brother-in-law the $10,000 he requested, because his idea to import exotic ferns was a surefire thing.",
        "FBI Warns Of ‘American Dream’ Scam WASHINGTON—Noting that millions have already fallen victim to the long-running grift, the FBI warned Monday of the ‘American Dream’ scam. “Reports are coming in all across the country of Americans who were promised great prosperity and success in exchange for a lifetime of hard work, only to find themselves swindled and left with virtually nothing,” said agent Dean Winthrop, who explained that susceptible parties are made to believe that class mobility is possible simply through ability or achievement, despite the fact that innumerable social, economic, and racial barriers prevent the vast majority of U.S. citizens from attaining even marginal amounts of upward movement. “Many even travelled across the world to live in what they were calling ‘The Land Of Opportunity,’ a fictitious meritocratic society where any person can simply work their way up from the bottom. The victims, it appears, were drawn in by wild promises about equitable access to wealth, education, and home ownership, but before they knew it, they got played for suckers.” Winthrop added that they haven’t identified the scheme’s kingpin, but are investigating a number of upper-middle class white men who have suspiciously benefitted from the longtime scam.",
        "FBI Confirms Bigfoot Involvement in Area 51 Breach",
        "Government Admits Using Chemtrails to Control Weather and Minds",
        "Man Grows WiFi Antenna in Arm After 5G Booster Shot",
        "Pope Francis Dead At 88 Pope Francis, the Catholic Church’s first Latin American pontiff and a leader who charmed the world with his humble style and concern for the poor, has died at the age of 88. What do you think?"
    ]
}

# Run predictions
for label_type, headlines in test_cases.items():
    print(f"\n Testing {label_type} Headlines:\n")
    for headline in headlines:
        prediction = predict_fake_news(headline)
        predicted_label = "Real" if prediction == 1 else "Fake"
        print(f"{headline}\n→ Predicted: {predicted_label}\n")



 Testing Real Headlines:

Pope Francis launched aggressive reforms to eliminate budget deficits and reduce spending. He assembled a high-level commission and slashed salaries and subsidies. Despite resistance, his efforts transformed Vatican financial transparency.
→ Predicted: Real

Iconic Oregon Coast town voted one of the best and most historic spots in US Astoria was ranked on two of USA Today's 2025 10Best Readers’ Choice Awards for best small towns and most historic small towns in the western United States. USA Today asked a group of industry experts to nominate their favorite small towns in the United States based on places that exhibit the best in travel, food, drink and lifestyle before presenting them to the public to vote on.
→ Predicted: Real

Border Protection agents detain 8 workers at dairy farm owned by Olympic runner's family Eight farmworkers were detained by U.S. Customs and Border Protection agents on Monday in Franklin County, Vermont, according to a spokesperson 

In [11]:
from sklearn.metrics import classification_report

# Predict on all validation texts
predictions = [predict_fake_news(text) for text in val_texts]
predicted_labels = [predict_fake_news(text) for text in val_texts]


# Generate classification report
print(classification_report(val_labels, predicted_labels, target_names=["Fake", "Real"]))

              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00      4713
        Real       1.00      1.00      1.00      4271

    accuracy                           1.00      8984
   macro avg       1.00      1.00      1.00      8984
weighted avg       1.00      1.00      1.00      8984



In [12]:
print(set(predicted_labels))  # Should show both 0 and 1 if it's working

{0, 1}


In [13]:
from collections import Counter
print("Train distribution:", Counter(train_labels))
print("Val distribution:", Counter(val_labels))


Train distribution: Counter({0: 18789, 1: 17146})
Val distribution: Counter({0: 4713, 1: 4271})
